# Normalization pipeline

Die Quelldaten sind aus meinem ersten RAG-Projekt und wurden mit fruendlicher Genehmigung von einer Webseite gescrappt.

**Was zu tun ist:**
- Daten von Marketingaussagen und Firmengeschichte befreien
- Die Specs normalisieren

Das meiste werde ich mit einem LLM machen. Die Ausgangsdaten werden stichprobenartig kontrolliert. Ziel ist ein funktionierendes und korrektes retrival, welches nicht zwingend mit der realen Welt (korrekte technische Daten z.B.) übereinstimmen muss. In einem Realworldprojekt ist das selbstverständlich etwas anderes.

Wärend der Bereinigung werden die Daten in Specs und Descriptions aufgeteilt.

In [1]:
import os
import json
import time
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

df = pd.read_json('./products_raw.json')

api_key = os.getenv('MISTRAL_API_KEY')
model = 'mistral-medium-2508'
client = Mistral(api_key=api_key)

descs_promt = open('./descs_agent.md', 'r').read()
specs_promt = open('./specs_agent.md', 'r').read()
summs_promt = open('./summs_agent.md').read()

descs_chunks = []
specs_chunks = []
summs_chunks = []

Die Beschreibungen werden über ein LLM bereinigt. Der Systempromt liegt als md-File vor. 

### Funktionen

- **agent_request**: Allgemeine Retrivalfunktion. Fallbackfunktion (wegen fehlenden Specs integriert)
- **format_spec**: Bereitet Embeddingdocs für Specs auf

In [2]:
def agent_request(promt, content, fallback):

    if content == '[]': content = fallback
    
    response = client.chat.complete(
        model = model,
        messages = [
            {
                'role': 'system',
                'content': promt
            },
            {
                'role': 'user',
                'content': content,
            }
        ],
        response_format = {"type": "json_object"}
    )

    return response

def format_spec(group_name, specs_list):
    if not specs_list:
        return ''
    
    if '-' in group_name:
        label, unit = group_name.rsplit('-', 1)
        header = f"{label} ({unit})"
    else:
        header = group_name
    
    spec_parts = []

    for key, value in specs_list.items():
        readable_key = key.replace('_', ' ')
        spec_parts.append(f"{readable_key}: {value}")

    return f"{header} - {', '.join(spec_parts)}"

# Erstellung der Chunks

Die Daten werden an ein LLM geliefert und wir erhalten nur die erzeugen Artefarkte. Diese werden in des Schema der Chunks integriert:

In [ ]:
for index, row in tqdm(df.iterrows(), total=len(df)):

    summs_response = agent_request(summs_promt, json.dumps(row.to_dict(), ensure_ascii=False), False)
    descs_response = agent_request(descs_promt, row['description'], False)
    specs_response = agent_request(specs_promt, json.dumps(row['specs'], ensure_ascii=False), row['description'])

    summs_result = json.loads(summs_response.choices[0].message.content)                              
    descs_result = json.loads(descs_response.choices[0].message.content)
    specs_result = json.loads(specs_response.choices[0].message.content)

    # Debug only
    # print(summs_result)
    # print(descs_result)
    # print(specs_result)

    category = summs_result['category']
    manufacturer = summs_result['manufacturer']
    title = summs_result['title']

    product_metadata = {
        'product_id': row['id'],
        'product_title': title,
        'product_url': row['url'],
        'product_category': category,
        'product_manufacturer': manufacturer,
        'product_price': row['price'],
        'product_sku': row['order_number']
    }

    summs_chunks.append({
        'document': summs_result['summary'],
        'metadata': {
            'chunk_id': f"{row['id']}_summ_0",
            'chunk_type': 'description',
            **product_metadata
        }
    })

    for i, pararagraph in enumerate(descs_result):
        descs_chunks.append({
            'document': pararagraph,
            'metadata': {
                'chunk_id': f"{row['id']}_desc_{i}",
                'chunk_type': 'description',
                **product_metadata
            }
        })

    for i, (group_name, specs_list) in enumerate(specs_result.items()):
        if specs_list:
            specs_chunks.append({
                'document': format_spec(group_name, specs_list),
                'metadata': {
                    'chunk_id': f"{row['id']}_spec_{i}",
                    'chunk_type': 'specs',
                    **product_metadata
                }
            })

    # Testing only
    # if index == 65: break

summs_df = pd.DataFrame(summs_chunks)
descs_df = pd.DataFrame(descs_chunks)
specs_df = pd.DataFrame(specs_chunks)

  1%|          | 1/151 [00:18<47:01, 18.81s/it]

{'summary': 'Der Kirsch LABO-288 PRO-ACTIVE ist ein Laborkühlschrank mit einem Nutzvolumen von 280 Litern und einer Temperaturregelung von ca. 0 bis +15 °C. Der Kühlschrank verfügt über eine statisch belüftete, geräuscharme und energiesparende Kältemaschine, die hermetisch gekapselt ist. Die automatische Abtauung wird durch zeitlich begrenzte und thermische Umkehr des Kältemittelkreislaufs überwacht. Der Innenraum besteht aus glattem Aluminium mit farbloser Schutzbeschichtung und bietet eine gleichmäßige Temperierung durch Umluftkühlung mit einem Querstromgebläse, das beim Öffnen der Tür automatisch abschaltet. Der Laborkühlschrank ist mit einem akustischen und optischen Alarmsignal bei abweichenden Temperaturen ausgestattet und verfügt über eine USB-Schnittstelle sowie einen potentialfreien Kontakt zum Anschluss an die zentrale Leittechnik. Diese Merkmale machen den Kirsch LABO-288 PRO-ACTIVE zu einem zuverlässigen Partner für die Kühlung in Labor, Klinik und im gesamten Gesundheitswe

  1%|▏         | 2/151 [00:39<48:48, 19.65s/it]

{'summary': 'Der Kirsch FROSTER LABEX-530 ULTIMATE ist ein Laborgefrierschrank mit 500 Litern Nutzvolumen und Temperaturregelung von -10°C bis -30°C. Der Gefrierschrank ist speziell für die Lagerung von leicht entzündlichen und explosiven Stoffen konzipiert und verfügt über einen explosionsgeschützten Innenraum, der die Sicherheitsanforderungen der Zone 2 erfüllt und mit einem TÜV-Zertifikat zertifiziert ist. Mit einer PRO-Active-Steuerung, automatischer Abtauung und einer 70 mm starken Isolierung bietet er höchste Sicherheit und Effizienz. Der Innenraum besteht aus poliertem Aluminium mit einer farblosen Schutzbeschichtung und ist mit fünf kunststoffbeschichteten Rosten ausgestattet. Der Gefrierschrank verfügt über ein optisches und akustisches Alarmsignal bei abweichenden Temperaturen und eine USB-Schnittstelle zur Datenauslese für die PC-gestützte Temperaturdokumentation. Mit einem Touch-Display, Zugangskontrolle mit Protokoll und grafischer Anzeige des Temperaturverlaufs ist er ide

  2%|▏         | 3/151 [00:59<49:10, 19.93s/it]

...

{'summary': 'Der Kirsch FROSTER MED-95 PRO-ACTIVE ist ein Medikamentengefrierschrank mit einem Nutzvolumen von 95 Litern und einer Temperaturregelung von -5°C bis -25°C. Der Gefrierschrank ist mit einer PRO-ACTIVE Steuerungselektronik ausgestattet, die eine permanente Überwachung der Leistungsdaten und Alarmierung bei Abweichungen ermöglicht. Zudem verfügt er über eine USB-Schnittstelle zur Temperaturdokumentation und eine Schnellfrosteinrichtung. Der FROSTER MED-95 ist besonders servicefreundlich durch seine frontseitige Blende und leicht zu reinigende Folientastatur. Der Innenraum besteht aus schlagfestem Kunststoff mit zwei Rosten und einem Schubfach. Der Gefrierschrank ist ideal für die sichere Lagerung von Medikamenten und Blutkonserven in medizinischen Einrichtungen.', 'category': 'Medikamentenkühlschrank', 'manufacturer': 'Kirsch', 'title': 'FROSTER MED-95 PRO-ACTIVE'}
['Der Kirsch FROSTER MED-95 ist ein Medikamentengefrierschrank mit PRO-Active-Steuerung, der durch seine geräus

 38%|███▊      | 58/151 [16:18<26:03, 16.81s/it]

{'summary': 'Der Kirsch LABEX-468 PRO-ACTIVE ist ein Laborkühlschrank, speziell für die Lagerung von leicht entzündlichen und explosiven Stoffen konzipiert. Mit einem Nutzvolumen von 460 Litern und einem Temperaturbereich von 0 bis +15°C bietet er eine sichere Umgebung für Chemikalien und andere gefährliche Materialien. Der Schrank ist mit einem explosionsgeschützten Innenraum ausgestattet, der die Sicherheitsanforderungen der Zone 2 erfüllt und durch ein TÜV-Zertifikat zündquellfrei ist. Die Kältemaschine ist geräuscharm, energiesparend und hermetisch gekapselt, was eine hohe Temperaturkonstanz bei geringem Energieverbrauch gewährleistet. Der LABEX-468 verfügt über eine automatische Abtauung, eine elektronische Temperatursteuerung und zahlreiche Sicherheitsfeatures wie akustische und optische Alarme. Mit seiner robusten Bauweise und hochwertigen Materialien ist dieser Laborkühlschrank ideal für den Einsatz in Laboren, Kliniken und im Gesundheitswesen.', 'category': 'Laborkühlschrank',

 39%|███▉      | 59/151 [16:31<24:01, 15.66s/it]

{'summary': 'Der Kirsch MED-720 ULTIMATE ist ein Medikamentenkühlschrank mit PRO-Active-Steuerung und DIN 58345 Normierung. Mit einem Fassungsvermögen von 700 Litern und zwölf Schubfächern bietet er eine enorme Staukapazität für Medikamente. Der Kühlschrank verfügt über eine Umluftkühlung und eine Temperaturregelung von +2°C bis +15°C. Besonders hervorzuheben sind das Touch-Display, die Zugangskontrolle mit Protokoll und die grafische Anzeige des Temperaturverlaufs. Der MED-720 ULTIMATE ist ideal für die sichere und übersichtliche Lagerung von Medikamenten in medizinischen Einrichtungen.', 'category': 'Medikamentenkühlschrank', 'manufacturer': 'Kirsch', 'title': 'MED-720 ULTIMATE'}
['Der Kirsch MED-720 ist ein nach DIN 58345 genormter Medikamentenkühlschrank mit einer PRO-Active-Steuerung, die eine permanente Überwachung der Leistungsdaten ermöglicht und bei Abweichungen alarmiert. Die Temperatursteuerung erfolgt präzise durch zwei serienmäßige PT-1000 Fühler.', 'Der Kühlschrank bietet

## Quality Checks

Wir müssen zuletzt noch in die Chunks schauen ob sie die wichtigsten Rahmenbedinungen einhalten. Vor allem die Länge da die meisten Models eine beschränkte Anzahl an Tokes verarbeiten kann und die darf von den Chunks nicht überschritten werden.

In [ ]:
def stats(df, treshhold, name):

    df['length'] = df['document'].str.len()
    small = df[df['length'] < treshhold]

    print(f'=== {name} ===')
    print(f"{df['length'].describe()}")
    print(f'TOTAL CHUNKS: {len(df)}')
    print(f'SHORT CHUNKS: {len(small)}')

    for idx, row in small.iterrows():
        
        print(f"{row['metadata']['product_title']}")
        print(f"Document: {row['document']}")
        print(f"Length: {row['length']}")
        print(f'--- --- ---')

    print('\n')


In [ ]:
stats(summs_df, 200, 'Summs')
stats(descs_df, 200, 'Descriptions')
stats(specs_df, 100, 'Specs')

An dieser Stelle wurden weitere Verbesserungen an den Promts vorgenommen und getestet... Mit den Specs muss noch eine andere Strategie ausgedacht werden. Jetzt erstmal mit den kleinen Chunks probieren. 

## Speichern

In [ ]:
summs_df.to_json('summs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
descs_df.to_json('descs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
specs_df.to_json('specs_chunks.jsonl', orient='records', lines=True, force_ascii=False)